In [2]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('default')
  
# %matplotlib inline: only draw static
# images in the notebookº
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Carga de información

In [3]:
laliga2122 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('H:/Mi unidad/Diplomado - Introducción a la Ciencia de Datos UC/MANE - MCD - Análisis de datos en Phyton/Semana 3/LigasEspanolas6T/LaLiga16 17.xlsx', )

## Creando dataframes

In [5]:
import pandas as pd

# Supongamos que ya tienes definidos los DataFrames laliga2122, laliga2021, laliga1920, etc.

laliga = pd.concat([laliga2122.dropna(how='all'), laliga2021.dropna(how='all'), laliga1920.dropna(how='all'), laliga1819.dropna(how='all'), laliga1718.dropna(how='all'), laliga1617.dropna(how='all')]).loc[:, ['Wk', 'Date', 'Local', 'Score', 'Visitante', 'Attendance']]
laliga['Attendance'] = laliga['Attendance'] * 1000
laliga['Torneo'] = 'Primera división'


## Creando nuevas columnas

In [6]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Total goles partido']= laliga['Goles Local']+laliga['Goles Visitante']


In [7]:
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)



## Función para crear matriz con puntos obtenidos de equipo i contra equipo j (i filas, j columnas)

In [8]:
def puntos(equipo1,equipo2,data):
    usos = data[((data['Local']==equipo1)&(data['Visitante']==equipo2))|((data['Local']==equipo2)&(data['Visitante']==equipo1))]
    puntos = 0
    for i in range(len(usos)):
        ganador = str(usos.iloc[i]['Ganador'])
        if ganador == equipo1:
            puntos += 3
        if ganador == 'Empate':
            puntos +=1
    return puntos

Creamos un dataframe con los nombres de los equipos que han jugado en laliga las últimas 6 temporadas

In [9]:
agrup_equip = laliga[laliga['Ganador']!= 'Empate'].groupby(['Ganador'])['Wk'].count().reset_index()[['Ganador']]
agrup_equip

,Ganador
0,Alavés
1,Athletic Club
2,Atlético Madrid
3,Barcelona
4,Betis
5,Celta Vigo
6,Cádiz
7,Eibar
8,Elche
9,Espanyol


Separamos estos mismos equipos que han jugado en una lista y la ordenamos alfabéticamente con el método sorted

In [10]:
equipos = sorted(laliga['Local'].unique().tolist())
equipos

['Alavés',
 'Athletic Club',
 'Atlético Madrid',
 'Barcelona',
 'Betis',
 'Celta Vigo',
 'Cádiz',
 'Eibar',
 'Elche',
 'Espanyol',
 'Getafe',
 'Girona',
 'Granada',
 'Huesca',
 'La Coruña',
 'Las Palmas',
 'Leganés',
 'Levante',
 'Mallorca',
 'Málaga',
 'Osasuna',
 'Rayo Vallecano',
 'Real Madrid',
 'Real Sociedad',
 'Sevilla',
 'Sporting Gijón',
 'Valencia',
 'Valladolid',
 'Villarreal']

Creamos la matriz buscada

In [11]:
for equipo in equipos:
    agrup_equip[equipo] = agrup_equip.apply(lambda x : puntos(x['Ganador'],equipo,laliga),axis=1)

Cambiamos el nombre de la primera columna para la visualización

In [12]:
agrup_equip = agrup_equip.rename(columns={'Ganador':'Equipo1\Equipo2'})
agrup_equip

,Equipo1\Equipo2,Alavés,Athletic Club,Atlético Madrid,Barcelona,Betis,Celta Vigo,Cádiz,Eibar,Elche,...,Málaga,Osasuna,Rayo Vallecano,Real Madrid,Real Sociedad,Sevilla,Sporting Gijón,Valencia,Valladolid,Villarreal
0,Alavés,0,17,6,5,12,13,4,15,6,...,10,4,6,6,11,8,4,12,13,21
1,Athletic Club,14,0,12,6,23,18,6,20,7,...,7,14,7,4,15,16,3,12,6,18
2,Atlético Madrid,27,21,0,11,27,27,12,23,12,...,12,24,12,9,23,17,6,26,16,15
3,Barcelona,29,27,20,0,28,19,2,26,12,...,7,17,6,18,32,28,6,25,18,29
4,Betis,21,11,6,7,0,22,10,10,7,...,12,22,8,12,13,8,2,18,7,10
5,Celta Vigo,22,15,6,13,10,0,5,12,7,...,4,12,5,2,10,11,4,11,5,13
6,Cádiz,7,6,0,8,1,5,0,6,4,...,0,0,3,5,0,1,0,6,2,5
7,Eibar,12,5,5,2,16,15,0,0,0,...,7,3,3,4,10,11,3,18,8,13
8,Elche,6,4,0,0,4,4,7,6,0,...,0,3,3,2,0,4,0,3,2,5
9,Espanyol,17,11,11,3,12,12,4,8,1,...,10,8,4,6,11,6,4,8,4,11


Revisamos los resultados del Alavés contra el Athletic club:


In [13]:
laliga[((laliga['Local']=='Alavés')&(laliga['Visitante']=='Athletic Club'))|(
(laliga['Local']=='Athletic Club')&(laliga['Visitante']=='Alavés')
)].groupby('Ganador')['Wk'].count().reset_index()

,Ganador,Wk
0,Alavés,4
1,Athletic Club,3
2,Empate,5


In [14]:
puntos('Alavés','Athletic Club',laliga),puntos('Athletic Club','Alavés',laliga)

(17, 14)